## Differential Privacy - Simple Database Queries

The database is going to be a VERY simple database with only one boolean column. Each row corresponds to a person. Each value corresponds to whether or not that person has a certain private attribute (such as whether they have a certain disease, or whether they are above/below a certain age). We are then going to learn how to know whether a database query over such a small database is differentially private or not - and more importantly - what techniques we can employ to ensure various levels of privacy

#### Create a Simple Database
To do this, initialize a random list of 1s and 0s (which are the entries in our database). Note - the number of entries directly corresponds to the number of people in our database.

In [30]:
import torch
# the number of entries in our DB / this of it as number of people in the DB
num_entries = 5000
db = torch.rand(num_entries) > 0.5
db

tensor([1, 1, 1,  ..., 1, 1, 1], dtype=torch.uint8)

## Generate Parallel Databases
> "When querying a database, if I removed someone from the database, would the output of the query change?". 

In order to check for this, we create "parallel databases" which are simply databases with one entry removed. 

We'll create a list of every parallel database to the one currently contained in the "db" variable. Then, create a helper function which does the following:
- creates the initial database (db)
- creates all parallel databases

In [32]:
def create_parallel_db(db, remove_index):
    return torch.cat((db[0:remove_index], db[remove_index+1:]))

In [33]:
def create_parallel_dbs(db):
    parallel_dbs = list()
    for i in range(len(db)):
        pdb = create_parallel_db(db, i)
        parallel_dbs.append(pdb)
    return parallel_dbs

In [34]:
def create_db_and_parallels(num_entries):
    # generate dbs and parallel dbs on the fly
    db = torch.rand(num_entries) > 0.5
    pdbs = create_parallel_dbs(db)
    
    return db, pdbs

In [44]:
db, pdbs = create_db_and_parallels(10)
pdbs
print("Real database:", db)
print("Size of real DB", db.size())
print("A sample parallel DB", pdbs[0])
print("Size of parallel DB", pdbs[0].size())

Real database: tensor([1, 1, 1, 0, 0, 0, 0, 0, 0, 0], dtype=torch.uint8)
Size of real DB torch.Size([10])
A sample parallel DB tensor([1, 1, 0, 0, 0, 0, 0, 0, 0], dtype=torch.uint8)
Size of parallel DB torch.Size([9])


# Towards Evaluating The Differential Privacy of a Function

Intuitively, we want to be able to query our database and evaluate whether or not the result of the query is leaking "private" information. 
> This is about evaluating whether the output of a query changes when we remove someone from the database. Specifically, we want to evaluate the *maximum* amount the query changes when someone is removed (maximum over all possible people who could be removed). 

To find how much privacy is leaked, we'll iterate over each person in the database and **measure** the difference in the output of the query relative to when we query the entire database. 

Just for the sake of argument, let's make our first "database query" a simple sum. Aka, we're going to count the number of 1s in the database.

In [53]:
db, pdbs = create_db_and_parallels(200)
def query(db):
    return db.sum()

In [60]:
query(db)

tensor(106)

In [61]:
# the output of the parallel dbs is different from the db query
query(pdbs[1])


tensor(106)

In [62]:
full_db_result = query(db)
print(full_db_result)

tensor(106)


In [67]:
sensitivity = 0
sensitivity_scale = []
for pdb in pdbs:
    pdb_result = query(pdb)
    db_distance = torch.abs(pdb_result - full_db_result)
    if(db_distance > sensitivity):
        sensitivity_scale.append(db_distance)
        sensitivity = db_distance

In [68]:
sensitivity

tensor(1)

#### Sensitivity
> The maximum amount the query changes when removing an individual from the DB.


# Evaluating the Privacy of a Function

The difference between each parallel db's query result and the query result for the real database and its max value (which was 1) is called "sensitivity". It corresponds to the function we chose for the query. The "sum" query will always have a sensitivity of exactly 1. We can also calculate sensitivity for other functions as well.

Let's calculate sensitivity for the "mean" function.

In [75]:
def sensitivity(query, num_entries=1000):
    db, pdbs = create_db_and_parallels(num_entries)
    
    full_db_result = query(db)
    
    max_distance = 0
    for pdb in pdbs:
        # for each parallel db, execute the query (sum, or mean, ..., etc)
        pdb_result = query(pdb)
        db_distance = torch.abs(pdb_result - full_db_result)
        
        if (db_distance > max_distance):
            max_distance = db_distance

    return max_distance 

In [76]:
# our query is now the mean
def query(db):
    return db.float().mean()

In [77]:

sensitivity(query)

tensor(0.0005)

Wow! That sensitivity is WAY lower. Note the intuition here. 
>"Sensitivity" is measuring how sensitive the output of the query is to a person being removed from the database. 

For a simple sum, this is always 1, but for the mean, removing a person is going to change the result of the query by rougly 1 divided by the size of the database. Thus, "mean" is a VASTLY less "sensitive" function (query) than SUM.

# Calculating L1 Sensitivity For Threshold

TO calculate the sensitivty for the "threshold" function: 

- First compute the sum over the database (i.e. sum(db)) and return whether that sum is greater than a certain threshold.
- Then, create databases of size 10 and threshold of 5 and calculate the sensitivity of the function. 
- Finally, re-initialize the database 10 times and calculate the sensitivity each time.

In [78]:
def query(db, threshold=5):
    """
    Query that adds a threshold of 5, and returns whether sum is > threshold or not.
    """
    return (db.sum() > threshold).float()

In [86]:
for i in range(10):
    sens = sensitivity(query, num_entries=10)
    print(sens)

0
tensor(1.)
0
0
0
0
0
0
0
tensor(1.)


# A Basic Differencing Attack

Sadly none of the functions we've looked at so far are differentially private (despite them having varying levels of sensitivity). The most basic type of attack can be done as follows.

Let's say we wanted to figure out a specific person's value in the database. All we would have to do is query for the sum of the entire database and then the sum of the entire database without that person!

## Performing a Differencing Attack on Row 10 (How privacy can fail)

We'll construct a database and then demonstrate how one can use two different sum queries to explose the value of the person represented by row 10 in the database (note, you'll need to use a database with at least 10 rows)

In [91]:
db, _ = create_db_and_parallels(100)

In [92]:
db

tensor([0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1,
        0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
        0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
        1, 0, 1, 0], dtype=torch.uint8)

In [93]:
# create a parallel db with that person (index 10) removed
pdb = create_parallel_db(db, remove_index=10)

In [94]:
pdb

tensor([0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
        1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
        0, 1, 0], dtype=torch.uint8)

In [95]:
# differencing attack using sum query
sum(db) - sum(pdb)

tensor(1, dtype=torch.uint8)

In [96]:
# a differencing attack using mean query
sum(db).float() /len(db) - sum(pdb).float() / len(pdb)

tensor(0.0051)

In [102]:
# differencing using a threshold
(sum(db).float() > 50) - (sum(pdb).float() > 50)

tensor(1, dtype=torch.uint8)

# Local Differential Privacy

Differential privacy always requires a form of randommess or noise added to the query to protect from things like Differencing Attacks.
To explain this, let's look at Randomized Response.

### Randomized Response (Local Differential Privacy)

Let's say I have a group of people I wish to survey about a very taboo behavior which I think they will lie about (say, I want to know if they have ever committed a certain kind of crime). I'm not a policeman, I'm just trying to collect statistics to understand the higher level trend in society. So, how do we do this? One technique is to add randomness to each person's response by giving each person the following instructions (assuming I'm asking a simple yes/no question):

- Flip a coin 2 times.
- If the first coin flip is heads, answer honestly
- If the first coin flip is tails, answer according to the second coin flip (heads for yes, tails for no)!

Thus, each person is now protected with "plausible deniability". If they answer "Yes" to the question "have you committed X crime?", then it might becasue they actually did, or it might be because they are answering according to a random coin flip. Each person has a high degree of protection. Furthermore, we can recover the underlying statistics with some accuracy, as the "true statistics" are simply averaged with a 50% probability. Thus, if we collect a bunch of samples and it turns out that 60% of people answer yes, then we know that the TRUE distribution is actually centered around 70%, because 70% averaged with a 50% (a coin flip) is 60% which is the result we obtained. 

However, it should be noted that, especially when we only have a few samples, this comes at the cost of accuracy. This tradeoff exists across all of Differential Privacy. 

> NOTE: **The greater the privacy protection (plausible deniability) the less accurate the results. **

Let's implement this local DP for our database before!

The main goal is to: 
* Get the most accurate query with the **greatest** amount of privacy
* Greatest fit with trust models in the actual world, (don't waste trust)

Let's implement local differential privacy:

In [118]:
db, pdbs = create_db_and_parallels(100)
db

tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1,
        0, 1, 0, 0], dtype=torch.uint8)

In [119]:
def query(db):
    true_result = torch.mean(db.float())
    
    # local differential privacy is adding noise to data: replacing some 
    # of the values with random values
    first_coin_flip = (torch.rand(len(db)) > 0.5).float()
    second_coin_flip = (torch.rand(len(db)) > 0.5).float()
    
    # differentially private DB ... 
    augmented_db = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip
    
    # the result is skewed if we do:
    # torch.mean(augmented_db.float())
    # we remove the skewed average that was the result of the differential privacy
    dp_result = torch.mean(augmented_db.float()) * 2 - 0.5
    
    return dp_result, true_result

In [120]:
db, pdbs = create_db_and_parallels(10)
private_result, true_result = query(db)
print(f"Without noise {private_result}")
print(f"With noise: {true_result}")

Without noise 0.7000000476837158
With noise: 0.30000001192092896


In [124]:
# Increasing the size of the dateset
db, pdbs = create_db_and_parallels(100)
private_result, true_result = query(db)
print(f"Without noise {private_result}")
print(f"With noise: {true_result}")

Without noise 0.42000001668930054
With noise: 0.4699999988079071


In [126]:
# Increasing the size of the dateset even further
db, pdbs = create_db_and_parallels(1000)
private_result, true_result = query(db)
print(f"Without noise {private_result}")
print(f"With noise: {true_result}")

Without noise 0.5099999904632568
With noise: 0.5210000276565552


As we have seen,
> The more data we have the more the noise will tend to not affect the output of the query

# Varying Amounts of Noise

We are going to augment the randomized response query to allow for varying amounts of randomness to be added. To do this, we bias the coin flip to be higher or lower and then run the same experiment. 

We'll  need to both adjust the likelihood of the first coin flip AND the de-skewing at the end (where we create the "augmented_result" variable).


In [130]:
# Noise < 0.5 sets the likelihood that the coin flip will be heads, and vice-versa.
noise = 0.2

true_result = torch.mean(db.float())
# let's add the noise to data: replacing some of the values with random values
first_coin_flip = (torch.rand(len(db)) > noise).float()
second_coin_flip = (torch.rand(len(db)) > 0.5).float()

# differentially private DB ... 
augmented_db = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip

# since the result will be skewed if we do: torch.mean(augmented_db.float())
# we'll remove the skewed average above by doing below:
dp_result = torch.mean(augmented_db.float()) * 2 - 0.5

sk_result = augmented_db.float().mean()
print('True result:', true_result)
print('Skewed result:', sk_result)
print('De-skewed result:', dp_result)

True result: tensor(0.5210)
Skewed result: tensor(0.5140)
De-skewed result: tensor(0.5280)


In [131]:
def query(db, noise=0.2):
    """Default noise(0.2) above sets the likelihood that the coin flip will be heads"""
    true_result = torch.mean(db.float())

    # local diff privacy is adding noise to data: replacing some 
    # of the values with random values
    first_coin_flip = (torch.rand(len(db)) > noise).float()
    second_coin_flip = (torch.rand(len(db)) > 0.5).float()

    # differentially private DB ... 
    augmented_db = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip

    # the result is skewed if we do:
    # torch.mean(augmented_db.float())
    # we remove the skewed average that was the result of the differential privacy
    sk_result = augmented_db.float().mean()
    private_result = ((sk_result / noise ) - 0.5) * noise / (1 - noise)

    return private_result, true_result

In [132]:
# test varying noise
db, pdbs = create_db_and_parallels(10)
private_result, true_result = query(db, noise=0.2)
print(f"Without noise {private_result}")
print(f"With noise: {true_result}")


Without noise 0.25
With noise: 0.30000001192092896


In [133]:
# Increasing the size of the dateset even further
db, pdbs = create_db_and_parallels(100)
private_result, true_result = query(db, noise=0.4)
print(f"Without noise {private_result}")
print(f"With noise: {true_result}")

Without noise 0.7333332300186157
With noise: 0.6399999856948853


In [134]:
# Increasing the size of the dateset even further
db, pdbs = create_db_and_parallels(10000)
private_result, true_result = query(db, noise=0.8)
print(f"Without noise {private_result}")
print(f"With noise: {true_result}")


Without noise 0.5264999866485596
With noise: 0.5004000067710876


From the analysis above, with more data, its easier to protect privacy with noise. It becomes a lot easier to learn about general characteristics in the DB because the algorithm has more data points to look at and compare with each other.

So differential privacy mechanisms has helped us filter out any information unique to individual data entities and try to let through information that is consistent across multiple different people in the dataset. 
> The larger the dataset, the easier it is to protect privacy. 

# The Formal Definition of Differential Privacy

The previous method of adding noise was called "Local Differentail Privacy" because we added noise to each datapoint individually. This is necessary for some situations wherein the data is SO sensitive that individuals do not trust noise to be added later. However, it comes at a very high cost in terms of accuracy. 

However, alternatively we can add noise AFTER data has been aggregated by a function. This kind of noise can allow for similar levels of protection with a lower affect on accuracy. However, participants must be able to trust that no-one looked at their datapoints _before_ the aggregation took place. In some situations this works out well, in others (such as an individual hand-surveying a group of people), this is less realistic.

Nevertheless, global differential privacy is incredibly important because it allows us to perform differential privacy on smaller groups of individuals with lower amounts of noise. Let's revisit our sum functions.

In [40]:
db, pdbs = create_db_and_parallels(100)

def query(db):
    return torch.sum(db.float())

def M(db):
    query(db) + noise

query(db)

tensor(40.)

So the idea here is that we want to add noise to the output of our function. We actually have two different kinds of noise we can add - Laplacian Noise or Gaussian Noise. However, before we do so at this point we need to dive into the formal definition of Differential Privacy.

![alt text](dp_formula.png "Title")

_Image From: "The Algorithmic Foundations of Differential Privacy" - Cynthia Dwork and Aaron Roth - https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf_

This definition does not _create_ differential privacy, instead it is a measure of how much privacy is afforded by a query M. Specifically, it's a comparison between running the query M on a database (x) and a parallel database (y). As you remember, parallel databases are defined to be the same as a full database (x) with one entry/person removed.

Thus, this definition says that FOR ALL parallel databases, the maximum distance between a query on database (x) and the same query on database (y) will be e^epsilon, but that occasionally this constraint won't hold with probability delta. Thus, this theorem is called "epsilon delta" differential privacy.

# Epsilon

Let's unpack the intuition of this for a moment. 

Epsilon Zero: If a query satisfied this inequality where epsilon was set to 0, then that would mean that the query for all parallel databases outputed the exact same value as the full database. As you may remember, when we calculated the "threshold" function, often the Sensitivity was 0. In that case, the epsilon also happened to be zero.

Epsilon One: If a query satisfied this inequality with epsilon 1, then the maximum distance between all queries would be 1 - or more precisely - the maximum distance between the two random distributions M(x) and M(y) is 1 (because all these queries have some amount of randomness in them, just like we observed in the last section).

# Delta

Delta is basically the probability that epsilon breaks. Namely, sometimes the epsilon is different for some queries than it is for others. For example, you may remember when we were calculating the sensitivity of threshold, most of the time sensitivity was 0 but sometimes it was 1. Thus, we could calculate this as "epsilon zero but non-zero delta" which would say that epsilon is perfect except for some probability of the time when it's arbitrarily higher. Note that this expression doesn't represent the full tradeoff between epsilon and delta.

# How To Add Noise for Global Differential Privacy

Global Differential Privacy adds noise to the output of a query.
We'll add noise to the output of our query so that it satisfies a certain epsilon-delta differential privacy threshold.

There are two kinds of noise we can add 
- Gaussian Noise
- Laplacian Noise. 

Generally speaking Laplacian is better, but both are still valid. Now to the hard question...

### How much noise should we add?

The amount of noise necessary to add to the output of a query is a function of four things:

- the type of noise (Gaussian/Laplacian)
- the sensitivity of the query/function
- the desired epsilon (ε)
- the desired delta (δ)

Thus, for each type of noise we're adding, we have different way of calculating how much to add as a function of sensitivity, epsilon, and delta.

Laplacian noise is increased/decreased according to a "scale" parameter b. We choose "b" based on the following formula.

`b = sensitivity(query) / epsilon`

In other words, if we set b to be this value, then we know that we will have a privacy leakage of <= epsilon. Furthermore, the nice thing about Laplace is that it guarantees this with delta == 0. There are some tunings where we can have very low epsilon where delta is non-zero, but we'll ignore them for now.

### Querying Repeatedly

- if we query the database multiple times - we can simply add the epsilons (Even if we change the amount of noise and their epsilons are not the same).

# Create a Differentially Private Query

Let's create a query function which sums over the database and adds just the right amount of noise such that it satisfies an epsilon constraint. query will be for "sum" and for "mean". We'll use the correct sensitivity measures for both.

In [108]:
epsilon = 0.001

In [77]:
import numpy as np

In [78]:
db, pdbs = create_db_and_parallels(100)

In [79]:
db

tensor([0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
        1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1,
        1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,
        0, 1, 0, 1], dtype=torch.uint8)

In [80]:
def sum_query(db):
    return db.sum()

In [109]:
def laplacian_mechanism(db, query, sensitivity):
    beta = sensitivity / epsilon
    noise = torch.tensor(np.random.laplace(0, beta, 1))
    
    return query(db) + noise

In [111]:
laplacian_mechanism(db, sum_query, 0.01)

tensor([46.2211], dtype=torch.float64)

In [112]:
def mean_query(db):
    return torch.mean(db.float())

In [117]:
laplacian_mechanism(db, mean_query, 1)

tensor([413.9655], dtype=torch.float64)

# Differential Privacy for Deep Learning

So what does all of this have to do with Deep Learning? Well, these mechanisms form the core primitives for how Differential Privacy provides guarantees in the context of Deep Learning. 

### Perfect Privacy
> "a query to a database returns the same value even if we remove any person from the database".

In the context of Deep Learning, we have a similar standard. 

> Training a model on a dataset should return the same model even if we remove any person from the dataset.

Thus, we've replaced "querying a database" with "training a model on a dataset". In essence, the training process is a kind of query. However, one should note that this adds two points of complexity which database queries did not have:

    1. do we always know where "people" are referenced in the dataset?
    2. neural models rarely never train to the same output model, even on identical data

The answer to (1) is to treat each training example as a single, separate person. Strictly speaking, this is often overly zealous as some training examples have no relevance to people and others may have multiple/partial (consider an image with multiple people contained within it). Thus, localizing exactly where "people" are referenced, and thus how much your model would change if people were removed, is challenging.

The answer to (2) is also an open problem. To solve this, lets look at PATE.

## Scenario: A Health Neural Network

You work for a hospital and you have a large collection of images about your patients. However, you don't know what's in them. You would like to use these images to develop a neural network which can automatically classify them, however since your images aren't labeled, they aren't sufficient to train a classifier.

However, being a cunning strategist, you realize that you can reach out to 10 partner hospitals which have annotated data. It is your hope to train your new classifier on their datasets so that you can automatically label your own. While these hospitals are interested in helping, they have privacy concerns regarding information about their patients. Thus, you will use the following technique to train a classifier which protects the privacy of patients in the other hospitals.

- 1) You'll ask each of the 10 hospitals to train a model on their own datasets (All of which have the same kinds of labels)
- 2) You'll then use each of the 10 partner models to predict on your local dataset, generating 10 labels for each of your datapoints
- 3) Then, for each local data point (now with 10 labels), you will perform a DP query to generate the final true label. This query is a "max" function, where "max" is the most frequent label across the 10 labels. We will need to add laplacian noise to make this Differentially Private to a certain epsilon/delta constraint.
- 4) Finally, we will retrain a new model on our local dataset which now has labels. This will be our final "DP" model.

So, let's walk through these steps. I will assume you're already familiar with how to train/predict a deep neural network, so we'll skip steps 1 and 2 and work with example data. We'll focus instead on step 3, namely how to perform the DP query for each example using toy data.

So, let's say we have 10,000 training examples, and we've got 10 labels for each example (from our 10 "teacher models" which were trained directly on private data). Each label is chosen from a set of 10 possible labels (categories) for each image.

In [3]:
import numpy as np

In [4]:
num_teachers = 10 # we're working with 10 partner hospitals
num_examples = 10000 # the size of OUR dataset
num_labels = 10 # number of lablels for our classifier

In [5]:
# fake predictions
fake_preds = (
    np.random.rand(
        num_teachers, num_examples
    ) * num_labels).astype(int).transpose(1,0)

In [6]:
fake_preds[:,0]


array([8, 1, 2, ..., 2, 2, 5])

In [9]:
# Step 3: Perform a DP query to generate the final true label/outputs,
# Use the argmax function to find the most frequent label across all 10 labels,
# Then finally add some noise to make it differentially private.

new_labels = list()
for an_image in fake_preds:
    # count the most frequent label the hospitals came up with
    label_counts = np.bincount(an_image, minlength=num_labels)

    epsilon = 0.1
    beta = 1 / epsilon

    for i in range(len(label_counts)):
        # for each label, add some noise to the counts
        label_counts[i] += np.random.laplace(0, beta, 1)

    new_label = np.argmax(label_counts)
    
    new_labels.append(new_label)

In [10]:
# new_labels
new_labels[:10]

[2, 2, 2, 2, 0, 5, 4, 0, 0, 4]

# PATE Analysis

In [15]:
# lets say the hospitals came up with these outputs... 9, 9, 3, 6 ..., 2
labels = np.array([9, 9, 3, 6, 9, 9, 9, 9, 8, 2])
counts = np.bincount(labels, minlength=10)
print(counts)
query_result = np.argmax(counts)
query_result


[0 0 1 1 0 0 1 0 1 6]


9

If every hospital says the result is 9, then we have very low sensitivity.
We could remove a person, from the dataset, and the query results still is 9,
then we have not leaked any information. 
Core assumption: The same patient was not present at any of this two hospitals.

Removing any one of this hospitals, acts as a proxy to removing one person, which means that if we do remove one hospital, the query result should not be different.



In [16]:
from syft.frameworks.torch.differential_privacy import pate

In [28]:
num_teachers, num_examples, num_labels = (100, 100, 10)
# generate fake predictions/labels
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int)
indices = (np.random.rand(num_examples) * num_labels).astype(int) # true answers

preds[:,0:10] *= 0

# perform PATE to find the data depended epsilon and data independent epsilon
data_dep_eps, data_ind_eps = pate.perform_analysis(
    teacher_preds=preds, 
    indices=indices, 
    noise_eps=0.1, 
    delta=1e-5
)
print('Data Independent Epsilon', data_ind_eps)
print('Data Dependent Epsilon', data_dep_eps)

assert data_dep_eps < data_ind_eps


Data Independent Epsilon 11.756462732485115
Data Dependent Epsilon 1.52655213289881


In [29]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 1.52655213289881


In [21]:
preds[:,0:50] *= 0

In [22]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5, moments=20)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 411.5129254649703
Data Dependent Epsilon: 9.219308825046408


# Where to Go From Here


Read:
    - Algorithmic Foundations of Differential Privacy: https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf
    - Deep Learning with Differential Privacy: https://arxiv.org/pdf/1607.00133.pdf
    - The Ethical Algorithm: https://www.amazon.com/Ethical-Algorithm-Science-Socially-Design/dp/0190948205
   
Topics:
    - The Exponential Mechanism
    - The Moment's Accountant
    - Differentially Private Stochastic Gradient Descent

Advice:
    - For deployments - stick with public frameworks!
    - Join the Differential Privacy Community
    - Don't get ahead of yourself - DP is still in the early days

# Application of DP in Private Federated Learning

DP works by adding statistical noise either at the input level or output level of the model so that you can mask out individual user contribution, but at the same time gain insight into th overall population without sacrificing privacy.

> Case: Figure out average money one has in their pockets.
We could go and ask someone how much they have in their wallet. They pick a random number between -100 and 100. Add that to the real value, say $20 and a picked number of 100. resulting in 120. That way, we have no way to know what the actual amount of money in their wallet is.
When sufficiently large numbers of people submit these results, if we take the average, the noise will cancel out and we'll start seeing the true average.


Apart from statistical use cases, we can apply DP in Private Federated learning.

Suppose you want to train a model using distributed learning across a number of user devices. One way to do that is to get all the private data from the devices, but that's not very privacy friendly. 

Instead, we send the model from the server  back to the devices. The devices will then train the model
using their user data, and only send the privatized model updates back to the server.
Server will then aggregate the updates and make an informed decision of the overall model on the server.
As you do more and more rounds, slowly the model converges to the true population without 
private user data having to leave the devices.
If you increase the level of privacy, the model converges a bit slower and vice versa.


# Project:

For the final project for this section, you're going to train a DP model using this PATE method on the MNIST dataset, provided below.

In [23]:
import torchvision.datasets as datasets
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)

100.1%

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


113.5%

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz


100.4%

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz


180.4%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [14]:
train_data = mnist_trainset.train_data
train_targets = mnist_trainset.train_labels

/Users/atrask/anaconda/lib/python3.6/site-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/Users/atrask/anaconda/lib/python3.6/site-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [12]:
test_data = mnist_trainset.test_data
test_targets = mnist_trainset.test_labels

/Users/atrask/anaconda/lib/python3.6/site-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/atrask/anaconda/lib/python3.6/site-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
